In [1]:
import gensim as gs
import math
import numpy as np
import tqdm

dct_title = gs.corpora.Dictionary.load ("../result/dct_title.dump")

corpus = gs.corpora.MmCorpus('../result/corpus_title_only.mm')

PARAM_K1 = 1.5
PARAM_B = 0.75
EPSILON = 0.25

corp_size = 0
corp_size_words = 0
for doc in tqdm.tqdm (corpus):
    corp_size_words += sum (dict (doc).itervalues ())
    corp_size += 1

avgdl = float (corp_size_words) / corp_size

idfs = {}

summ_idf = 0


for word_id, freq in dct_title.dfs.iteritems ():
    idfs[word_id] = math.log(corp_size - freq + 0.5) - math.log(freq + 0.5)
    summ_idf += idfs[word_id]

average_idf = float (summ_idf) / len (dct_title.dfs)

for id in idfs.iterkeys ():
    idfs[id] = idfs[id] if idfs[id] >= 0 else EPSILON * average_idf

q_ids = []
queries = []
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").upper ().split ("\t")
        q_ids.append (int (q[0]))
        queries.append (dict (dct_title.doc2bow (q[1].split ())))

q_size = len (queries)

result = np.zeros ((q_size, corp_size))

100%|██████████| 582167/582167 [00:06<00:00, 83707.87it/s]


In [2]:
for doc_i, doc in tqdm.tqdm (enumerate (corpus)):
    doc = dict (doc)
    doc_keys = set (doc)
    doc_len = sum (doc.itervalues ())
    for q_i, q in enumerate (queries):
        score = 0
        for word in set (q.keys ()) & doc_keys:
            idf = idfs[word]
            score += (idf * doc[word] * (PARAM_K1 + 1)
                      / (doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len / avgdl)))
        result[q_i, doc_i] = score

582167it [1:25:50, 113.02it/s]


In [4]:
len (result[0,:])

582167

In [6]:
np.argsort (result[0,:])[-5:][::-1]

array([241765, 563677, 461894,  49847, 118027])

In [14]:
with open ("../result/result_bm25_aver_0_try_0_title_self.csv", "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for q_num, qid in tqdm.tqdm (enumerate (q_ids)):
        for doc in np.argsort (result[q_num, :])[-5:][::-1]:
            fout.write ("{},{}\n".format (qid, doc))

6311it [01:29, 70.60it/s]


In [ ]:
np.save ("bm25_title_result.dump", result)